In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import imodelsx.llm

import openai
openai.api_key_path = '/home/chansingh/.OPENAI_KEY'
outcome = 'iai'

df = pd.read_pickle("../data/data_clean.pkl")
n = df.shape[0]

llm = imodelsx.llm.get_llm('gpt-4-0314', repeat_delay=1)

# df[df.title.str.contains('pecarn', case=False)]
row = df[df.id==3971][['title', 'description', 'feature_names']].iloc[0]
row

# with pd.option_context("display.max_rows", None, "display.max_columns", None, "display.max_colwidth", 1000):
#     display(df[["id", "title", "feature_names", "info___mdcalc_url"]])

### General bias

In [ ]:
prompt = f'''### Question: Describe any potential issues with bias in the development of the following clinical decision instrument. If no issues are found, return "No issues".

- Title: {row["title"]}
- Description: {row["description"]}
- Predictor variables: {str(row["feature_names"])[1:-1]}

### Answer:'''
print(prompt)
print(llm(prompt))

### Feature bias

In [ ]:
Q = 'Describe any potential issues with bias in predictor variables of the following clinical decision instrument. If no issues are found, return "No issues".'
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {
        "role": "user",
        "content": f"""{Q}
Title: PECARN Pediatric Intra-Abdominal Injury (IAI) Algorithm
Description: Identifies children at very low risk of clinically important blunt abdominal injuries.
Predictor variables: 'Nausea/vomiting', 'Decreased breath sounds', 'Evidence of abdominal wall trauma or seatbelt sign', 'Abdominal tenderness', 'Ethniciy', 'Abdominal pain', 'Glasgow Coma Scale', 'Thoracic wall trauma'""",
    },
    {
        "role": "assistant",
        "content": """Potential issues with bias in predictor variables:
1. 'Ethnicity': fundamentally biased variable        
2. 'Abdominal tenderness': subjectivity in physician's assessment
3. 'Abdominal pain': subjectivity in patient's assessment
4. 'Glasgow Coma Scale': subjectivity in physician's assessment""",
    },
    # {"role": "user", "content": "Where was it played?"},
]

d_race = df[df.feature_names.apply(lambda x: "Race/Ethnicity" in x)]
for i in range(10):
    row = d_race.iloc[i]
    prompt = f"""{Q}
Title: {row["title"]}
Description: {row["description"]}
Predictor variables: {str(row["feature_names"])[1:-1]}"""
    messages[-1] = {"role": "user", "content": prompt}
    print(prompt)
    print(llm(messages))
    print("\n\n")